In [1]:
import functools
from collections import OrderedDict

# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
import sympy
from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.operators import I, Z
from qiskit.aqua.operators.list_ops import SummedOp
from qiskit.circuit.library import RealAmplitudes

In [2]:
M = [2, 6, 3]
K = [1, 4, 2]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [3]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "0000011001"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        key = key[::-1]
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    if solution_vector_correct(key) and execution_time(key) == d:
        return True
    return False


correct_machines = ['00', '01', '10']
machine_to_index = {'00': 2, '01': 1, '10': 0}


def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    if task1_machine in correct_machines and task2_machine in correct_machines and task3_machine in correct_machines:
        return True
    return False


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_time = time_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_time = time_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_time = time_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


def execution_cost(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    
    task1_cost = cost_matrix[task1_machine, 0] if task1_machine is not None else 0
    task2_cost = cost_matrix[task2_machine, 1] if task2_machine is not None else 0
    task3_cost = cost_matrix[task3_machine, 2] if task3_machine is not None else 0
    

    return task1_cost + task2_cost + task3_cost


def incorrect_machine_count(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])


    return (0 if k[0:2] in correct_machines else 1) \
         + (0 if k[2:4] in correct_machines else 1) \
         + (0 if k[4:6] in correct_machines else 1)

In [4]:
def create_Z(z_indices, n):
    return functools.reduce(lambda a, b : a ^ b, reversed([Z if i in z_indices else I for i in range(n)])) 


def get_summed_ops(coefficient_dict, n):
    ops = []
    offset = 0
    
    for summand, coefficient in coefficient_dict.items():
        coefficient = float(coefficient)
        if summand.is_Number:
            offset += float(summand)
        elif summand.is_Symbol:
            index = int(summand.name[1:])
            ops += [coefficient * create_Z([index], n)]
        elif summand.is_Mul:
            indices = [
                int(factor.base.name[1:]) 
                if factor.is_Pow
                else int(factor.name[1:]) 
                for factor 
                in summand.args
                if not factor.is_Pow or factor.exp % 2 == 1
            ]
            ops += [coefficient * create_Z(indices, n)]
        elif summand.is_Pow:
            indices = []
            if summand.exp % 2 == 1:
                indices += [int(summand.base.name[1:])]
            ops += [coefficient * create_Z(indices, n)]
        else:
            print('Error: unknown summand', summand)

    return SummedOp(ops).reduce(), offset

In [5]:
def to_ising(x):
    return .5 * (1 - x)


def get_cost_model(A, x):
    return sympy.simplify(sympy.expand(A * sum([
            cost_matrix[2, i] * (1 - to_ising(x[2 * i])) * (1 - to_ising(x[2 * i + 1]))
            + cost_matrix[1, i] * (1 - to_ising(x[2 * i])) * to_ising(x[2 * i + 1])
            + cost_matrix[0, i] * to_ising(x[2 * i]) * (1 - to_ising(x[2 * i + 1]))
#             + 10000 * to_ising(x[2 * i]) * to_ising(x[2 * i + 1])
            for i in range(0, 3)
    ])))


def get_machine_usage_model(B, x):
    return sympy.expand(B * sum([
        to_ising(x[2 * i]) * to_ising(x[2 * i + 1]) for i in range(0, 3)
    ]))


def _square_deadline_model(time_diff):
    result = []
    exprs = []
    for first_arg in time_diff.args:
        for second_arg in time_diff.args:
            non_zero = True
            expression = first_arg * second_arg
            
            changed = False
            for symbol in expression.free_symbols:
                if expression.find((.5 * (1 - symbol)) ** 2):
                    expression = expression.replace((.5 * (1 - symbol)) ** 2, .5 * (1 - symbol))
                    changed = True
                    
            if changed:
                exprs += [expression]
                continue         
                    
            for x_i in x:
                if x_i not in expression.free_symbols or expression.count(x_i) < 2:
                    continue

                isolated_expression = 1
                for factor in expression.args:
                    if x_i in factor.free_symbols:
                        isolated_expression *= factor

                at_minus_one = isolated_expression.evalf(subs={x_i: -1})
                at_one = isolated_expression.evalf(subs={x_i: 1})
                if at_minus_one.is_Number and at_minus_one < 1e-100 and at_one.is_Number and at_one < 1e-100:
                    non_zero = False
                    
            if non_zero:
                exprs += [expression]
    
    return sum(exprs)


def get_deadline_model(C, x):
    time_sum = sum([
                  time_matrix[2, i] * (1 - to_ising(x[2 * i])) * (1 - to_ising(x[2 * i + 1]))
                + time_matrix[1, i] * (1 - to_ising(x[2 * i])) * to_ising(x[2 * i + 1])
                + time_matrix[0, i] * to_ising(x[2 * i]) * (1 - to_ising(x[2 * i + 1]))
#                 + 10 * to_ising(x[2 * i]) * to_ising(x[2 * i + 1])
                for i in range(0, 3)
    ])
    slack_sum = 8 * to_ising(x[6]) + 4 * to_ising(x[7]) + 2 * to_ising(x[8]) + to_ising(x[9])
    time_constraint = C * (d - time_sum - slack_sum) ** 2
    
    return sympy.simplify(sympy.expand(time_constraint))

In [6]:
def create_hamiltonian(A, B, C):
    x = sympy.symbols(' '.join([f'x{i}' for i in range(10)]))
    
    cost_model = get_cost_model(A, x)
    machine_usage_model = get_machine_usage_model(B, x)
    deadline_model = get_deadline_model(C, x)
     
    model = cost_model + machine_usage_model + deadline_model
    
    coefficients = sympy.simplify(model).as_coefficients_dict()
    
    hamiltonian, offset = get_summed_ops(coefficients, len(x))
    
    return hamiltonian, offset

In [7]:
def compute_eigenvalues(quibit_op):
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 1024
    eigensolver = NumPyEigensolver(qubit_op, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print('state\t\ttime\tcost\tmachine use\tcorrect\teigenvalue')
    for eigenstate, eigenvalue in zip(eigensolver_result.eigenstates, eigensolver_result.eigenvalues):
        eigenstate, = eigenstate.sample().keys()
        eigenstate = eigenstate[::-1]
        eigenvalue = eigenvalue
        print(f'{eigenstate}\t{execution_time(eigenstate)}\t{execution_cost(eigenstate)}', end='')
        print(f'\t{incorrect_machine_count(eigenstate)}\t\t{is_correct(eigenstate)}\t{eigenvalue}')

In [8]:
A = 1
B = 20
C = 10

In [9]:
qubit_op, offset = create_hamiltonian(A, B, C)

In [10]:
compute_eigenvalues(qubit_op)

state		time	cost	machine use	correct	eigenvalue
1001100000	19.0	22.0	0		True	(-22.25+0j)
0010100000	19.0	23.0	0		True	(-21.25+0j)
0110100010	19.0	23.0	0		True	(-21.25+0j)
0101100100	19.0	24.0	0		True	(-20.25+0j)
0001100010	19.0	24.0	0		True	(-20.25+0j)
0000100001	19.0	24.0	0		True	(-20.25+0j)
0100100011	19.0	24.0	0		True	(-20.25+0j)
1010010110	19.0	25.0	0		True	(-19.25+0j)
1010000010	19.0	25.0	0		True	(-19.25+0j)
1000010111	19.0	26.0	0		True	(-18.25+0j)
1001011000	19.0	26.0	0		True	(-18.25+0j)
1001000100	19.0	26.0	0		True	(-18.25+0j)
1000000011	19.0	26.0	0		True	(-18.25+0j)
0010011000	19.0	27.0	0		True	(-17.25+0j)
0110000110	19.0	27.0	0		True	(-17.25+0j)
0010000100	19.0	27.0	0		True	(-17.25+0j)
0110011010	19.0	27.0	0		True	(-17.25+0j)
0100000111	19.0	28.0	0		True	(-16.25+0j)
0101011100	19.0	28.0	0		True	(-16.25+0j)
0101001000	19.0	28.0	0		True	(-16.25+0j)
0100011011	19.0	28.0	0		True	(-16.25+0j)
0001011010	19.0	28.0	0		True	(-16.25+0j)
0001000110	19.0	28.0	0		True	(-16.25+0j)
000001100

0101011110	21.0	28.0	0		False	(23.75+0j)
0101001010	21.0	28.0	0		False	(23.75+0j)
0100011101	21.0	28.0	0		False	(23.75+0j)
1010111000	17.0	9.0	1		False	(24.75+0j)
1010111100	21.0	9.0	1		False	(24.75+0j)
1001111110	21.0	10.0	1		False	(25.75+0j)
1000111001	17.0	10.0	1		False	(25.75+0j)
1000111101	21.0	10.0	1		False	(25.75+0j)
1001111010	17.0	10.0	1		False	(25.75+0j)
0010111010	17.0	11.0	1		False	(26.75+0j)
0110111100	17.0	11.0	1		False	(26.75+0j)
0010111110	21.0	11.0	1		False	(26.75+0j)
0100111101	17.0	12.0	1		False	(27.75+0j)
0000111111	21.0	12.0	1		False	(27.75+0j)
0000111011	17.0	12.0	1		False	(27.75+0j)
0001111100	17.0	12.0	1		False	(27.75+0j)
0101111110	17.0	12.0	1		False	(27.75+0j)
1110100110	21.0	15.0	1		False	(30.75+0j)
1110100010	17.0	15.0	1		False	(30.75+0j)
1101101000	21.0	16.0	1		False	(31.75+0j)
1101100100	17.0	16.0	1		False	(31.75+0j)
1100100111	21.0	16.0	1		False	(31.75+0j)
1100100011	17.0	16.0	1		False	(31.75+0j)
1011100011	21.0	18.0	1		False	(33.75+0j)
1110001010	21.0	19

1011001001	23.0	22.0	1		False	(157.75+0j)
1011011101	23.0	22.0	1		False	(157.75+0j)
1011010101	15.0	22.0	1		False	(157.75+0j)
1011000001	15.0	22.0	1		False	(157.75+0j)
1110111100	15.0	3.0	2		False	(158.75+0j)
0011011111	23.0	24.0	1		False	(159.75+0j)
0011010111	15.0	24.0	1		False	(159.75+0j)
1100111101	15.0	4.0	2		False	(159.75+0j)
0011000011	15.0	24.0	1		False	(159.75+0j)
1101111110	15.0	4.0	2		False	(159.75+0j)
0011001011	23.0	24.0	1		False	(159.75+0j)
0111001101	23.0	24.0	1		False	(159.75+0j)
0111011001	15.0	24.0	1		False	(159.75+0j)
0111000101	15.0	24.0	1		False	(159.75+0j)
1011111001	15.0	6.0	2		False	(161.75+0j)
0111111101	15.0	8.0	2		False	(163.75+0j)
0011111011	15.0	8.0	2		False	(163.75+0j)
1111101011	23.0	12.0	2		False	(167.75+0j)
1111100011	15.0	12.0	2		False	(167.75+0j)
1111001111	23.0	16.0	2		False	(171.75+0j)
1111011011	15.0	16.0	2		False	(171.75+0j)
1111000111	15.0	16.0	2		False	(171.75+0j)
1111111111	15	0	3		False	(175.75+0j)
1010100011	24.0	21.0	0		False	(226.75+0j)
100

1111011000	12.0	16.0	2		False	(501.75+0j)
1111000100	12.0	16.0	2		False	(501.75+0j)
1111111100	12	0	3		False	(505.75+0j)
1010100110	27.0	21.0	0		False	(616.75+0j)
1000100111	27.0	22.0	0		False	(617.75+0j)
1001101000	27.0	22.0	0		False	(617.75+0j)
0110101010	27.0	23.0	0		False	(618.75+0j)
0010101000	27.0	23.0	0		False	(618.75+0j)
0001101010	27.0	24.0	0		False	(619.75+0j)
0000101001	27.0	24.0	0		False	(619.75+0j)
0101101100	27.0	24.0	0		False	(619.75+0j)
0100101011	27.0	24.0	0		False	(619.75+0j)
1010001010	27.0	25.0	0		False	(620.75+0j)
1010011110	27.0	25.0	0		False	(620.75+0j)
1000011111	27.0	26.0	0		False	(621.75+0j)
1001001100	27.0	26.0	0		False	(621.75+0j)
1000001011	27.0	26.0	0		False	(621.75+0j)
1001010000	11.0	26.0	0		False	(621.75+0j)
0010001100	27.0	27.0	0		False	(622.75+0j)
0110010010	11.0	27.0	0		False	(622.75+0j)
0010010000	11.0	27.0	0		False	(622.75+0j)
0110001110	27.0	27.0	0		False	(622.75+0j)
0001001110	27.0	28.0	0		False	(623.75+0j)
0100010011	11.0	28.0	0		False	(623.75+0

In [11]:
seed = 10598
aqua_globals.random_seed = seed

max_trials = 1000
shots = 1000
reps = 2

entanglement = 'full'

In [12]:
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubit_op.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubit_op, ry, spsa)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result_vqe = vqe.run(quantum_instance)   
state_vector_vqe = result_vqe['eigenstate']
get_stats_for_result(state_vector_vqe)

most likely solution:  (array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0]), 292)
optimal:  140
correct solutions:  317
incorrect solutions:  683
correct configs:  5
incorrect configs:  32


In [13]:
spsa = SPSA(maxiter=max_trials)

qaoa = QAOA(qubit_op, optimizer=spsa, p=1)

# provider = IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = qaoa.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

most likely solution:  (array([0, 0, 1, 1, 0, 1, 1, 0, 0, 1]), 5)
optimal:  2
correct solutions:  23
incorrect solutions:  977
correct configs:  16
incorrect configs:  614
